# Imports

In [1]:
import json, random

In [2]:
from market import Market, Produto

# Static Values

In [3]:
CART: dict = {
    'Altura': 65,
    'Largura': 50,
    'Comprimento': 80,
    'PesoMaximo': 460*1000
}

CART['Capacidade'] = CART['Altura'] * CART['Largura'] * CART['Comprimento']

CART

{'Altura': 65,
 'Largura': 50,
 'Comprimento': 80,
 'PesoMaximo': 460000,
 'Capacidade': 260000}

In [4]:
MAX_MONEY: float = 2500.0
MAX_ITEMS: int = 20
MAX_CARTS: int = MAX_ITEMS*10
MAX_GENERATIONS: int = 100
BESTS: int = 15

# Funções

In [5]:
m: Market = Market(r"C:\Users\Arklok\Documents\Projetos\gen-algorithm\data\produtos.csv")

# Genético

### Geração de Indivíduos

In [6]:
def gerar_carrinho(tamanho: int = globals()['MAX_ITEMS']) -> list[Produto]:
    compra = []

    while len(compra) < tamanho:
        item: Produto = m.random()
        compra.append(item)
        
    return compra

In [7]:
c: list[Produto] = gerar_carrinho()
c

[<Produto(id=1615262>,
 <Produto(id=318906>,
 <Produto(id=324197>,
 <Produto(id=198166>,
 <Produto(id=479199>,
 <Produto(id=440145>,
 <Produto(id=289978>,
 <Produto(id=47384>,
 <Produto(id=715821>,
 <Produto(id=1618531>,
 <Produto(id=379833>,
 <Produto(id=541932>,
 <Produto(id=1609288>,
 <Produto(id=321143>,
 <Produto(id=1591037>,
 <Produto(id=190728>,
 <Produto(id=58351>,
 <Produto(id=457507>,
 <Produto(id=1433067>,
 <Produto(id=321143>]

### Geração de População Inicial

In [8]:
def gerar_populacao(tamanho: int = globals()['MAX_CARTS']) -> list[list[Produto]]:
    return [gerar_carrinho() for _ in range(tamanho)]

In [9]:
p: list[list[Produto]] = gerar_populacao()
p

[[<Produto(id=1111943>,
  <Produto(id=1606865>,
  <Produto(id=1497103>,
  <Produto(id=451162>,
  <Produto(id=457552>,
  <Produto(id=457548>,
  <Produto(id=1616960>,
  <Produto(id=1619139>,
  <Produto(id=431874>,
  <Produto(id=476990>,
  <Produto(id=794143>,
  <Produto(id=470253>,
  <Produto(id=1466988>,
  <Produto(id=158229>,
  <Produto(id=75455>,
  <Produto(id=13089>,
  <Produto(id=273343>,
  <Produto(id=309625>,
  <Produto(id=74949>,
  <Produto(id=169113>],
 [<Produto(id=1377637>,
  <Produto(id=50866>,
  <Produto(id=419635>,
  <Produto(id=1600697>,
  <Produto(id=419634>,
  <Produto(id=1615262>,
  <Produto(id=365815>,
  <Produto(id=329010>,
  <Produto(id=461589>,
  <Produto(id=318906>,
  <Produto(id=1616568>,
  <Produto(id=190826>,
  <Produto(id=13918>,
  <Produto(id=1571405>,
  <Produto(id=1618540>,
  <Produto(id=1376293>,
  <Produto(id=695249>,
  <Produto(id=321143>,
  <Produto(id=81476>,
  <Produto(id=162383>],
 [<Produto(id=117446>,
  <Produto(id=1624532>,
  <Produto(id=175398>,
 

### Avaliação

In [10]:
# def avaliar(individuo: list[Produto], valor_limite: float = globals()['MAX_MONEY']) -> float:
#     valor_compra: float = 0.0
    
#     for item in individuo:
#         valor_compra += item.price

#     if valor_compra > valor_limite:
#         return float('inf')
    
#     else:
#         return valor_compra

In [11]:
def avaliar(individuo: list[Produto], 
            valor_limite: float = globals()['MAX_MONEY'], 
            max_peso: float = CART['PesoMaximo'], 
            max_volume: float = CART['Capacidade'],
            max_itens: int = globals()['MAX_ITEMS']
            ) -> float:

    valor_compra: float = 0.0
    peso_total: float = 0.0
    volume_total: float = 0.0
    categorias: set = set()
    repeticoes: int = 0

    for item in individuo:
        valor_compra += item.price
        peso_total += item.weight
        volume_item = item.height * item.width * item.depth
        volume_total += volume_item
        
        categorias.add(item.category)
        
        # Contar repetições de produtos
        if individuo.count(item) > 1:
            repeticoes += 1

    # Penalidades
    nota = 0.0

    # Penalidade por exceder o limite de dinheiro
    if valor_compra > valor_limite:
        nota += (valor_compra - valor_limite) * 10  # Ajuste de penalidade
    
    # Penalidade por excesso de peso
    if peso_total > max_peso:
        nota += (peso_total - max_peso) * 15  # Ajuste de penalidade

    # Penalidade por excesso de volume
    if volume_total > max_volume:
        nota += (volume_total - max_volume) * 20  # Ajuste de penalidade

    # Penalidade por número excessivo de itens
    if len(individuo) > max_itens:
        nota += (len(individuo) - max_itens) * 5  # Ajuste de penalidade

    # Penalidade por repetição de produtos
    nota += repeticoes * 10  # Penalidade por cada produto repetido

    # Incentivo pela diversidade de categorias
    nota -= len(categorias) * 5  # Deduz nota pela diversidade de categorias

    # Final result is the total cost plus penalizations
    return valor_compra + nota


In [12]:
n: float = avaliar(c)
n

415.49000000000007

In [13]:
def mapear_notas(populacao: list[list[Produto]], order: str = 'asc') -> dict[float, list[Produto]]:
    notas: dict = {avaliar(individuo): individuo for individuo in populacao}
    
    if order == 'asc':
        return dict(sorted(notas.items()))
    
    else:
        return dict(sorted(notas.items(), reverse=True))

In [14]:
nt: dict[float, list[Produto]] = mapear_notas(p)
nt

{204.79000000000005: [<Produto(id=198166>,
  <Produto(id=1602362>,
  <Produto(id=169113>,
  <Produto(id=97199>,
  <Produto(id=479389>,
  <Produto(id=457550>,
  <Produto(id=332651>,
  <Produto(id=1376782>,
  <Produto(id=438171>,
  <Produto(id=379837>,
  <Produto(id=472881>,
  <Produto(id=302930>,
  <Produto(id=51141>,
  <Produto(id=470249>,
  <Produto(id=100533>,
  <Produto(id=461591>,
  <Produto(id=13627>,
  <Produto(id=1621539>,
  <Produto(id=1606865>,
  <Produto(id=44486>],
 239.11: [<Produto(id=541932>,
  <Produto(id=457552>,
  <Produto(id=271505>,
  <Produto(id=1624611>,
  <Produto(id=335996>,
  <Produto(id=78656>,
  <Produto(id=290020>,
  <Produto(id=350972>,
  <Produto(id=1591035>,
  <Produto(id=181458>,
  <Produto(id=198166>,
  <Produto(id=1433063>,
  <Produto(id=1440417>,
  <Produto(id=336142>,
  <Produto(id=568623>,
  <Produto(id=10480>,
  <Produto(id=1622212>,
  <Produto(id=1441479>,
  <Produto(id=365990>,
  <Produto(id=164887>],
 246.28999999999996: [<Produto(id=51141>,
  <P

### Seleção

In [15]:
def torneio(populacao: list[list[Produto]]) -> list[list[Produto]]:
    vencedores: list[list[Produto] | None] = []
    
    while len(vencedores) < 2:
        competidor_1: list[Produto] = random.choice(populacao)
        competidor_2: list[Produto] = random.choice(populacao)
                
        # Seleção do melhor individuo:
        if avaliar(competidor_1) < avaliar(competidor_2):
            vencedor: list[Produto] = competidor_1
        
        else:
            vencedor: list[Produto] = competidor_2
            
        if vencedor not in vencedores:
            vencedores.append(vencedor)
    
    return vencedores

In [16]:
v: list[list[Produto]] = torneio(p)
v

[[<Produto(id=1628300>,
  <Produto(id=232466>,
  <Produto(id=97185>,
  <Produto(id=379834>,
  <Produto(id=1628301>,
  <Produto(id=173710>,
  <Produto(id=182417>,
  <Produto(id=1556318>,
  <Produto(id=454739>,
  <Produto(id=240852>,
  <Produto(id=69291>,
  <Produto(id=711940>,
  <Produto(id=1471540>,
  <Produto(id=431736>,
  <Produto(id=169774>,
  <Produto(id=1433060>,
  <Produto(id=1616578>,
  <Produto(id=365815>,
  <Produto(id=431865>,
  <Produto(id=127976>],
 [<Produto(id=444216>,
  <Produto(id=1616567>,
  <Produto(id=1245489>,
  <Produto(id=1433067>,
  <Produto(id=463527>,
  <Produto(id=53077>,
  <Produto(id=348305>,
  <Produto(id=379837>,
  <Produto(id=1628304>,
  <Produto(id=1466988>,
  <Produto(id=165471>,
  <Produto(id=22595>,
  <Produto(id=240852>,
  <Produto(id=1624611>,
  <Produto(id=169774>,
  <Produto(id=1107472>,
  <Produto(id=112743>,
  <Produto(id=278608>,
  <Produto(id=25181>,
  <Produto(id=348305>]]

In [17]:
def eugenia(populacao: list[list[Produto]], order: str = 'asc') -> list[list[Produto]]:
    notas: dict[float, list[Produto]] = mapear_notas(populacao, order)
    melhores: list[list[Produto]] = list(notas.values())[:globals()['BESTS']]
    
    return melhores

In [18]:
e: list[list[Produto]] = eugenia(p)
e

[[<Produto(id=198166>,
  <Produto(id=1602362>,
  <Produto(id=169113>,
  <Produto(id=97199>,
  <Produto(id=479389>,
  <Produto(id=457550>,
  <Produto(id=332651>,
  <Produto(id=1376782>,
  <Produto(id=438171>,
  <Produto(id=379837>,
  <Produto(id=472881>,
  <Produto(id=302930>,
  <Produto(id=51141>,
  <Produto(id=470249>,
  <Produto(id=100533>,
  <Produto(id=461591>,
  <Produto(id=13627>,
  <Produto(id=1621539>,
  <Produto(id=1606865>,
  <Produto(id=44486>],
 [<Produto(id=541932>,
  <Produto(id=457552>,
  <Produto(id=271505>,
  <Produto(id=1624611>,
  <Produto(id=335996>,
  <Produto(id=78656>,
  <Produto(id=290020>,
  <Produto(id=350972>,
  <Produto(id=1591035>,
  <Produto(id=181458>,
  <Produto(id=198166>,
  <Produto(id=1433063>,
  <Produto(id=1440417>,
  <Produto(id=336142>,
  <Produto(id=568623>,
  <Produto(id=10480>,
  <Produto(id=1622212>,
  <Produto(id=1441479>,
  <Produto(id=365990>,
  <Produto(id=164887>],
 [<Produto(id=51141>,
  <Produto(id=339035>,
  <Produto(id=78656>,
  <Prod

### Crossover e Mutação

In [19]:
def cruzamento(pais: list[list[Produto]], chance: float = 0.8) -> list[list[Produto]]:
    pai_1: list[Produto] = pais[0]
    pai_2: list[Produto] = pais[1]

    probabilidade: float = round(random.uniform(0, 1), 2)

    if probabilidade < chance:
        pos: int = random.randint(0, len(pai_1))

        filho_1: list[Produto] = pai_1[:pos] + pai_2[pos:]
        filho_2: list[Produto] = pai_2[:pos] + pai_1[pos:]

        return [filho_1, filho_2]

    else:
        return [pai_1, pai_2]

In [20]:
f: list[list[Produto]] = cruzamento(v)
f

[[<Produto(id=1628300>,
  <Produto(id=232466>,
  <Produto(id=97185>,
  <Produto(id=379834>,
  <Produto(id=1628301>,
  <Produto(id=173710>,
  <Produto(id=182417>,
  <Produto(id=1556318>,
  <Produto(id=454739>,
  <Produto(id=240852>,
  <Produto(id=69291>,
  <Produto(id=711940>,
  <Produto(id=1471540>,
  <Produto(id=431736>,
  <Produto(id=169774>,
  <Produto(id=1433060>,
  <Produto(id=1616578>,
  <Produto(id=365815>,
  <Produto(id=25181>,
  <Produto(id=348305>],
 [<Produto(id=444216>,
  <Produto(id=1616567>,
  <Produto(id=1245489>,
  <Produto(id=1433067>,
  <Produto(id=463527>,
  <Produto(id=53077>,
  <Produto(id=348305>,
  <Produto(id=379837>,
  <Produto(id=1628304>,
  <Produto(id=1466988>,
  <Produto(id=165471>,
  <Produto(id=22595>,
  <Produto(id=240852>,
  <Produto(id=1624611>,
  <Produto(id=169774>,
  <Produto(id=1107472>,
  <Produto(id=112743>,
  <Produto(id=278608>,
  <Produto(id=431865>,
  <Produto(id=127976>]]

In [21]:
def mutar(populacao: list[list[Produto]], chance = 0.05) -> list[list[Produto]]: 
    mutados: list[list[Produto]] = []

    for individuo in populacao:
        for pos, alelo in enumerate(individuo):
            mutacao: float = round(random.uniform(0, 1), 3)

            if mutacao < chance:
                novo_gene: str = m.random(ignore_case=[alelo,]) 
                individuo: str = f'{individuo[:pos]}{novo_gene}{individuo[pos+1:]}'

        mutados.append(individuo)
    
    return mutados

In [22]:
mt: list[list[Produto]] = mutar(p)
mt

['[<Produto(id=11119Product Info\nid: 560779\nname: Snack Mucilon Tradicional 35g\nsku: 1267003\nprice: 4.49\nimage: https://static.paodeacucar.com/img/uploads/1/21/30495021.png\nbrand: Nestlé\ncategory: bebes-e-criancas\ndescription: nan\nweight: 39.0\nheight: 23.0\nwidth: 6.5\ndepth: 3.5\n3>, <Produto(id=1606865>, <Produto(id=1497103>, <Produto(id=451162>, <Produto(id=457552>, <Produto(id=457548>]Product Info\nid: 103465\nname: Detergente Líquido Cristal Limpol Squeeze 500ml\t\nsku: 2280806\nprice: 2.59\nimage: https://static.paodeacucar.com/img/uploads/1/358/25034358.jpg\nbrand: Limpol\ncategory: limpeza\ndescription: O Lava-Louças Limpol é eficiente no poder da limpeza, remoção da gordura e sujeiras e no combate aos maus odores indesejados da cozinha. Sua fórmula é dermatologicamente testada e contém glicerina, tornando-a mais suave para as mãos.\nweight: 527.6\nheight: 24.5\nwidth: 5.7\ndepth: 5.7\n[<Produto(id=1619139>, <Produto(id=431874>, <Produto(id=476990>, <Produto(id=794143

# Execução Principal

In [23]:
if __name__ == '__main__':
    g: int = 0
    
    while g < MAX_GENERATIONS:
        g+=1

        populacao_inicial: list[list[Produto]] = gerar_populacao()
        populacao_atual: list[list[Produto]] = populacao_inicial.copy()
        geracao: list[list[Produto]] = []

        while len(geracao) < MAX_CARTS - BESTS:
            notas: dict[float, list[Produto]] = mapear_notas(populacao_atual)
            
            pais: list[list[Produto]] = torneio(populacao_atual)
            filhos: list[list[Produto]] = cruzamento(pais)
            geracao.extend(filhos)
            
        geracao.extend(eugenia(populacao_atual))
        
        # print(f'Geração: {g} | Melhor: {notas[min(notas.keys())]}')
        populacao_atual: list[list[Produto]] = geracao.copy()

In [24]:
for item in eugenia(populacao_atual)[0]:
    print(str(item))
    item.show()

Product Info
id: 182417
name: Desodorante Aerosol GIOVANNA BABY Rosa 150ml
sku: 4731924
price: 14.29
image: https://static.paodeacucar.com/img/uploads/1/474/442474.jpg
brand: Giovanna Baby
category: cuidados-pessoais
description: nan
weight: 188.0
height: 18.0
width: 5.0
depth: 5.0

Product Info
id: 379880
name: NESTLÉ® PURINA® FRISKIES® Ração Úmida para Gatos Filhotes Carne ao molho 85g
sku: 1118091
price: 3.19
image: https://static.paodeacucar.com/img/uploads/1/704/20246704.jpg
brand: Purina
category: petshop
description: FRISKIES Filhotes Carne ao molho é um alimento úmido completo e balanceado destinado a filhotes de gatos de 1 a 12 meses e a fêmeas gestantes ou lactantes. Pele e pelos saudáveis graças aos níveis adequados de zinco e ácidos graxos essenciais. Visão clara e saudável devido à adição de vitamina A e taurina. Formulado com vitaminas e minerais que auxiliam nas defesas naturais. Músculos fortes e saudáveis devido à presença de fontes de proteínas de alta qualidade. Para